# Tests for HanTa English

Some simple tests to check whether everything is doing what it is suposed to do.

In [31]:
import sys
sys.path.insert(1, '..')
import HanoverTagger as ht #Do not import form the package but from the parent folder where the latest source file is found

tagger = ht.HanoverTagger(r'../morphmodel_en.pgz')

## A number of arbitrary examples

In [32]:
print(tagger.analyze('walking',taglevel=0))
print(tagger.analyze('walking',taglevel=1))
print(tagger.analyze('walking',taglevel=2))
print(tagger.analyze('walking',taglevel=3))

VBG
('walk', 'VBG')
('walk+ing', 'VBG')
('walk', [('walk', 'VB'), ('ing', 'SUF_ING')], 'VBG')


In [33]:
print(tagger.analyze('loving',taglevel=3))
print(tagger.analyze('unexpectedly',taglevel=3))
print(tagger.analyze('developed',taglevel=3))
print(tagger.analyze('honestly',taglevel=3))
print(tagger.analyze('men',taglevel=3))
print(tagger.analyze('women',taglevel=2))
print(tagger.analyze('actualities',taglevel=3))
print(tagger.analyze('actualities',taglevel=1))

('love', [('lov', 'VB_VAR'), ('ing', 'SUF_ING')], 'VBG')
('unexpectedly', [('un', 'PREF_NEG'), ('expect', 'VB'), ('ed', 'SUF_VB_N'), ('ly', 'SUF_RB')], 'RB')
('develop', [('develop', 'VB'), ('ed', 'SUF_VB_N')], 'VBN')
('honestly', [('honest', 'JJ'), ('ly', 'SUF_RB')], 'RB')
('man', [('men', 'NN_VAR')], 'NNS')
('woman', 'NNS')
('actuality', [('actual', 'JJ'), ('iti', 'SUF_JJN_VAR'), ('es', 'SUF_NN_S')], 'NNS')
('actuality', 'NNS')


In [34]:
print(tagger.tag_word('love'))
print(tagger.tag_word('school'))
print(tagger.tag_word('eating'))

[('NN', -8.903083013526377), ('VB', -9.944943475986376)]
[('NN', -7.875083013526378), ('VB', -12.601743475986376)]
[('VBG', -10.581802033139532), ('NN', -13.380383013526377)]


In [35]:
print(tagger.analyze('simplest',taglevel=3))
print(tagger.analyze('simplest',taglevel=1))
print(tagger.analyze('reliable',taglevel=3))
print(tagger.analyze('heavily',taglevel=3))
print(tagger.analyze('unhappiest',taglevel=3))
print(tagger.analyze('intimidating',taglevel=3))

('simple', [('simpl', 'JJ_VAR'), ('est', 'SUF_JJ_T')], 'JJT')
('simple', 'JJT')
('reliable', [('reli', 'VB_VAR'), ('able', 'SUF_VBJJ')], 'JJ')
('heavily', [('heavi', 'JJ_VAR'), ('ly', 'SUF_RB')], 'RB')
('unhappy', [('unhappiest', 'JJ_VAR')], 'JJR')
('intimidating', [('intimidating', 'NN')], 'NN')


In [36]:
tagger.tag_word('intimidating')

[('NN', -29.585544593704444),
 ('JJ', -30.146018749111022),
 ('VBG', -31.07004127090513),
 ('NNS', -33.409677300707365),
 ('VB', -34.41452704525451)]

In [37]:
import nltk

#sent = "Tackling the entire kitchen can be an intimidating task, so here’s a manageable list of things to clean, ingredients to check, equipment to organize and more."
sent = "Tackling the entire kitchen can be an intimidating task, so here's a manageable list of things to clean, ingredients to check, equipment to organize and more."

words = nltk.word_tokenize(sent)

print(tagger.tag_sent(words,taglevel = 0))
print('----')
print(tagger.tag_sent(words,taglevel = 1))
print('----')
print(tagger.tag_sent(words,taglevel = 2))
print('----')
print(tagger.tag_sent(words,taglevel = 3))

['VBG', 'AT', 'JJ', 'NN', 'MD', 'BE', 'AT', 'JJ', 'NN', ',', 'QL', 'RB', 'BEZ', 'AT', 'JJ', 'NN', 'IN', 'NNS', 'TO', 'VB', ',', 'NNS', 'IN', 'NN', ',', 'NN', 'TO', 'VB', 'CC', 'AP', '.']
----
[('Tackling', 'tackl', 'VBG'), ('the', 'the', 'AT'), ('entire', 'entire', 'JJ'), ('kitchen', 'kitchen', 'NN'), ('can', 'can', 'MD'), ('be', 'be', 'BE'), ('an', 'a', 'AT'), ('intimidating', 'intimidating', 'JJ'), ('task', 'task', 'NN'), (',', ',', ','), ('so', 'so', 'QL'), ('here', 'here', 'RB'), ("'s", "'s", 'BEZ'), ('a', 'a', 'AT'), ('manageable', 'manageable', 'JJ'), ('list', 'list', 'NN'), ('of', 'of', 'IN'), ('things', 'thing', 'NNS'), ('to', 'to', 'TO'), ('clean', 'clean', 'VB'), (',', ',', ','), ('ingredients', 'ingredient', 'NNS'), ('to', 'to', 'IN'), ('check', 'check', 'NN'), (',', ',', ','), ('equipment', 'equipment', 'NN'), ('to', 'to', 'TO'), ('organize', 'organize', 'VB'), ('and', 'and', 'CC'), ('more', 'more', 'AP'), ('.', '.', '.')]
----
[('Tackling', 'tackl+ing', 'VBG'), ('the', 'th

## Evaluate tagging and lemmatising on train data

This is ofcourse not a proper evaluation as we use the training data to evaluate the program. However, if we don't get good results here, something went wrong for sure. The numbers we get here might give a kind of upperbound for what we can ecpect from a real evaluation.

Evaluation of tagging and lemmatisation is not a trivial task. First, the evaluation data might have been tagged according to some othe tagging scheme. A lot of different decisions can be made about the tags and lemmata for many unclear cases. Moreover most data sets contain a large number of errors. Often these originate form the tagger/stemmer/lemmatiser that was used for the initial annotation and were overlooked in the manual correction phases.  

In [38]:
def load(lines):
    data = []
    sent = []
    lastsentnr = 1
    for line in lines:
        (sentnr,word,lemma,stem,tag,morphemes,stemsub) = line.split('\t')
        if sentnr != lastsentnr:
            if len(sent) > 0:
                data.append((sentnr,sent))
            sent = []
            lastsentnr = sentnr
        sent.append((word,lemma,tag))
    return data

In [23]:
import codecs

datafile = codecs.open(r"labeledmorph_en.csv", "r","utf-8")

morphdata = []
for line in datafile:
    if not line.startswith('-1'):
        morphdata.append(line)
    

In [24]:
testdata = load(morphdata)

### Evaluate the POS Tagging

In [39]:
def tag_evaluate(sents):
    correct = 0
    nr = 0

    for snr,sent in sents:
        ws = [w for (w,l,c) in sent]
        cs = [c for (w,l,c) in sent]
        pred_cs = tagger.tag_sent(ws,taglevel = 0)
        for i in range(len(ws)): 
            nr += 1
            if cs[i] == pred_cs[i]:
                correct += 1
            #else:
            #    c = cs[i].split('(')[0]
            #    pred_c = pred_cs[i].split('(')[0]
            #    if c != pred_c:
            #        print(snr,'\t',' '.join(ws))
            #        print(ws[i],cs[i],pred_cs[i])
            #        print()
        if nr%50 == 0:
            print(correct/nr,end='\r')
    return correct/nr

In [40]:
tag_evaluate(testdata)

0.9795492729768678

### Evaluate lemmatisation

In [41]:
def lemma_evaluate(sents):
    correct = 0
    nr = 0

    for snr,sent in sents:
        ws = [w for (w,l,c) in sent]
        ls = [l for (w,l,c) in sent]
        pred_ls = [l for _,l,_ in tagger.tag_sent(ws,taglevel = 1)]
        for i in range(len(ws)): 
            nr += 1
            if ls[i].lower() == pred_ls[i].lower():
                correct += 1
            #else:
            #    print(snr,'\t',' '.join(ws))
            #    print(ws[i],ls[i],pred_ls[i])
            #    print()
        if nr%50 == 0:
            print(correct/nr,end='\r')
    return correct/nr

In [42]:
lemma_evaluate(testdata)


0.9982592764600222